## Previous result

In [1]:
!pwd

/Users/yibeihu/bestbuy_project/notebooks/yibei


In [2]:
import sys
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

sys.version

sys.path.append("../../")

# from src.ad_hmm import sku_predict
# from src.ad_stl_model import stl_model
from src.utils import *

In [3]:
%load_ext autoreload 
%autoreload 

In [4]:
train_test = pd.read_csv("../../data/train_validation_marker.csv")
train_test["SALES_DATE"] = pd.to_datetime(train_test["SALES_DATE"])
print (train_test.shape)

train_test["if_promo"] = (~train_test["PROMO_PRICE"].isin(["?"])).astype("int")
train_test["if_competitor"] = (~train_test["COMPETITOR_PRICE"].isin(["?"])).astype("int")

train_test["min_sales_price"] = pd.to_numeric(np.where(train_test.if_promo>0, train_test.PROMO_PRICE,\
                                                       train_test.RETAIL_PRICE))

train_test["comp_price_clean"] = pd.to_numeric(np.where(train_test.if_competitor>0,train_test.COMPETITOR_PRICE, np.nan))
train_test["price_delta"] = np.where(train_test.if_competitor>0, train_test.min_sales_price - train_test.comp_price_clean,\
                                   -train_test.min_sales_price)

(846632, 15)


In [5]:
train_test_pivot = train_test.pivot(index='SALES_DATE', columns='Encoded_SKU_ID',\
                   values=['DAILY_UNITS',"if_promo","if_competitor","min_sales_price","price_delta",\
                          "validation","validation_clean"])
train_test_pivot.columns = train_test_pivot.columns.swaplevel(0, 1)


In [6]:
import pickle
with open('../../data/stl_results_v1.pkl', 'rb') as f:
    all_preds = pickle.load(f)

In [7]:
y_pred_df = pd.DataFrame()
for i in range(len(all_preds)):
    if all_preds[i]!= None:
        y_pred_df = pd.concat([y_pred_df,all_preds[i][1]])

In [8]:
validation = train_test[(train_test.validation==True) & (train_test.validation_clean==True)]
y_act = validation[["Encoded_SKU_ID","SALES_DATE","DAILY_UNITS"]].rename(columns = {"DAILY_UNITS":"actual"})
y_act.head()

,Encoded_SKU_ID,SALES_DATE,actual
100,96,2022-07-27,2
218,187,2022-07-29,0
420,297,2022-07-29,6
652,372,2022-07-27,1
831,412,2022-07-26,4


### Null， STL/MSTL， HMM and holt winters

In [9]:
y_pred_null = pd.read_csv("../../data/null_prediction_validation_result_v0.csv").drop(columns = "Unnamed: 0")
y_pred_null["SALES_DATE"] = pd.to_datetime(y_pred_null["SALES_DATE"])

In [10]:
y_pred_stl = y_pred_df.reset_index()[["SALES_DATE","STL_prediction","Encoded_SKU_ID"\
                        ]].rename(columns={"STL_prediction":"predicted"})

y_pred_mstl = y_pred_df.reset_index()[["SALES_DATE","MSTL_prediction","Encoded_SKU_ID"\
                        ]].rename(columns={"MSTL_prediction":"predicted"})

# y_act = y_pred_df.reset_index()[["SALES_DATE","actual","Encoded_SKU_ID"\
#                         ]]
y_act["actual"] = pd.to_numeric(y_act["actual"])

In [11]:
y_pred_holt = pd.read_csv("../../data/ES_pred.csv").drop(columns=["Unnamed: 0"])
y_pred_holt["SALES_DATE"] = pd.to_datetime(y_pred_holt["SALES_DATE"])
y_pred_holt.head()

,SALES_DATE,predicted,Encoded_SKU_ID
0,2022-07-25,2.006318,1
1,2022-07-27,2.536772,1
2,2022-07-29,2.003338,1
3,2022-07-31,2.569282,1
4,2022-07-26,2.104498,1


In [12]:
VERSION = "v2"
hmm_output = pd.read_csv(f"../../data/hmm_result_{VERSION}.csv",index_col=0)

### Join all the data

In [13]:
print("null",y_pred_null["Encoded_SKU_ID"].nunique())
print("stl",y_pred_stl["Encoded_SKU_ID"].nunique())
print("mstl",y_pred_mstl["Encoded_SKU_ID"].nunique())
print("holt",y_pred_holt["Encoded_SKU_ID"].nunique())
print("hmm",hmm_output["Encoded_SKU_ID"].nunique())

null 545
stl 503
mstl 503
holt 545
hmm 352


In [14]:
hmm_output1 = hmm_output.reset_index(names="SALES_DATE")
hmm_output1["SALES_DATE"] = pd.to_datetime(hmm_output1["SALES_DATE"])
y_pred_holt1 = y_pred_holt.rename(columns={"predicted":"predicted_holt"})

In [15]:
all_predictions = hmm_output1.merge(y_pred_df.reset_index(),on=["Encoded_SKU_ID","SALES_DATE"],how="outer"\
                                            ).merge(y_pred_null,on=["Encoded_SKU_ID","SALES_DATE"],how="outer",\
                                                   suffixes = ("_15","_null"),\
                                                   ).merge(y_pred_holt1,on=["Encoded_SKU_ID","SALES_DATE"],how="outer"\
                                                          ,suffixes=("","_holt")).drop(columns="actual")

In [16]:
all_predictions.head()

,SALES_DATE,predicted_15,predicted_30,predicted_45,predicted_60,predicted_90,predicted_max,Encoded_SKU_ID,STL_prediction,MSTL_prediction,predicted_null,predicted_holt
0,2022-07-25,1.0,1.0,1.0,0.00,2.00,3.00,1,1.708770,2.087032,1,2.006318
1,2022-07-26,1.0,1.0,2.0,3.00,0.45,2.00,1,1.642804,0.778320,1,2.104498
2,2022-07-27,2.0,0.0,3.0,2.00,2.55,1.00,1,1.464536,1.561164,1,2.536772
3,2022-07-28,0.0,1.0,0.0,0.45,0.45,2.45,1,1.454188,2.314384,1,2.601272
4,2022-07-29,3.0,2.0,1.0,2.00,2.55,2.00,1,1.738205,1.581958,1,2.003338


#### QC

In [17]:
all_predictions = all_predictions.rename(columns={"STL_prediction":"predicted_STL",\
                                                  "MSTL_prediction":"predicted_MSTL"})

In [18]:
all_predictions.isna().sum()

SALES_DATE           0
predicted_15      1351
predicted_30      1351
predicted_45      1351
predicted_60      1351
predicted_90      1351
predicted_max     1351
Encoded_SKU_ID       0
predicted_STL      312
predicted_MSTL     323
predicted_null       0
predicted_holt       0
dtype: int64

### Get RMSE for each columns

In [19]:
rmse_all_preds = pd.DataFrame()
first_flag = True
for col in all_predictions.columns:
    if "pred" in col:
        y_pred_i = all_predictions[["Encoded_SKU_ID","SALES_DATE",col\
                                   ]].rename(columns={col:"predicted"})
        y_pred_i["SALES_DATE"] = pd.to_datetime(y_pred_i["SALES_DATE"])
        rmse_pred_i = rmse_sku(y_act,y_pred_i)
        rmse_pred_i.columns = [f"{col_rmse}_{col.split('_')[-1]}" for col_rmse in rmse_pred_i.columns]
        if first_flag:
            rmse_all_preds = rmse_pred_i
            print (col,rmse_all_preds.shape)
            first_flag = False
        else:
            rmse_all_preds = rmse_all_preds.merge(rmse_pred_i, left_index = True, right_index = True,\
                                suffixes = ("",f"_{col.split('_')[-1]}"))
            print (col,rmse_all_preds.shape)

predicted_15 (545, 3)
predicted_30 (545, 6)
predicted_45 (545, 9)
predicted_60 (545, 12)
predicted_90 (545, 15)
predicted_max (545, 18)
predicted_STL (545, 21)
predicted_MSTL (545, 24)
predicted_null (545, 27)
predicted_holt (545, 30)


In [20]:
rmse_all_preds

,rmse_du_15,mean_du_15,pct_rmse_15,rmse_du_30,mean_du_30,pct_rmse_30,rmse_du_45,mean_du_45,pct_rmse_45,rmse_du_60,...,pct_rmse_STL,rmse_du_MSTL,mean_du_MSTL,pct_rmse_MSTL,rmse_du_null,mean_du_null,pct_rmse_null,rmse_du_holt,mean_du_holt,pct_rmse_holt
Encoded_SKU_ID,,,,,,,,,,,,,,,,,,,,,
82,4.722325,0.428571,11.018759,4.840344,0.428571,11.294136,1.102108,0.428571,2.571586,1.419381,...,17.664067,5.381107,0.428571,12.555916,3.605551,0.428571,8.412953,9.753826,0.428571,22.758928
568,4.097124,4.285714,0.955996,3.447670,4.285714,0.804456,3.652836,4.285714,0.852329,3.871231,...,0.815257,2.705275,4.285714,0.631231,5.070926,4.285714,1.183216,3.024439,4.285714,0.705702
317,3.766488,0.857143,4.394235,1.362770,0.857143,1.589899,1.784957,0.857143,2.082450,2.104417,...,1.744665,2.523250,0.857143,2.943792,1.069045,0.857143,1.247219,2.363641,0.857143,2.757581
95,3.668447,1.285714,2.853236,1.698949,1.285714,1.321405,3.991151,1.285714,3.104228,3.931648,...,0.927400,2.534372,1.285714,1.971178,1.309307,1.285714,1.018350,1.720612,1.285714,1.338254
340,3.668447,0.714286,5.135825,2.077172,0.714286,2.908041,2.329929,0.714286,3.261901,1.558387,...,1.257594,0.972092,0.714286,1.360928,0.925820,0.714286,1.296148,1.092959,0.714286,1.530142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,...,18.242177,9.913714,1.000000,9.913714,1.069045,1.000000,1.069045,19.176286,1.000000,19.176286
559,NaN,3.857143,NaN,NaN,3.857143,NaN,NaN,3.857143,NaN,NaN,...,0.803188,3.156090,3.857143,0.818246,3.207135,3.857143,0.831479,4.635827,3.857143,1.201881
560,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,...,NaN,NaN,1.000000,NaN,0.534522,1.000000,0.534522,0.827778,1.000000,0.827778


### Pick Best

In [21]:
rmse_cols = [col for col in rmse_all_preds.columns if "rmse_du" in col]
rmse_cols

['rmse_du_15',
 'rmse_du_30',
 'rmse_du_45',
 'rmse_du_60',
 'rmse_du_90',
 'rmse_du_max',
 'rmse_du_STL',
 'rmse_du_MSTL',
 'rmse_du_null',
 'rmse_du_holt']

In [22]:
(~rmse_all_preds[rmse_cols].isna()).sum()

rmse_du_15      352
rmse_du_30      352
rmse_du_45      352
rmse_du_60      352
rmse_du_90      352
rmse_du_max     352
rmse_du_STL     502
rmse_du_MSTL    503
rmse_du_null    545
rmse_du_holt    545
dtype: int64

In [23]:
rmse_all_preds["min_rmse"] = rmse_all_preds[rmse_cols].min(axis=1,skipna=True)

rmse_all_preds["argmin_period"] = np.argmin(rmse_all_preds[rmse_cols].values,axis=1)

rmse_all_preds["min_period"] = rmse_all_preds["argmin_period"].apply(lambda x: rmse_cols[x])

In [24]:
rmse_all_preds["min_rmse"].isna().sum()

0

In [25]:
rmse_all_preds["min_period"].value_counts()

rmse_du_15      222
rmse_du_STL      94
rmse_du_max      41
rmse_du_30       36
rmse_du_null     35
rmse_du_60       30
rmse_du_90       27
rmse_du_45       22
rmse_du_holt     21
rmse_du_MSTL     17
Name: min_period, dtype: int64

In [26]:
period_col_map= {
    "rmse_du_15":"predicted_15", "rmse_du_30":"predicted_30", "rmse_du_45":"predicted_45",\
    "rmse_du_60":"predicted_60", "rmse_du_90":"predicted_90", "rmse_du_max":"predicted_max",\
    "rmse_du_STL":"predicted_STL","rmse_du_MSTL":"predicted_MSTL","rmse_du_null":"predicted_null",\
    "rmse_du_holt":"predicted_holt"
}

In [27]:
y_pred_best = pd.DataFrame()
cnt = 0
for sku_id,best_period in zip(rmse_all_preds.index,rmse_all_preds["min_period"]):
    best_col = period_col_map[best_period]
    mask_sku = all_predictions.Encoded_SKU_ID==sku_id
    y_pred_best_sku = all_predictions.loc[mask_sku][["Encoded_SKU_ID","SALES_DATE",best_col\
                                                                  ]].rename(columns={best_col:"predicted"})
    y_pred_best = pd.concat([y_pred_best,y_pred_best_sku])

In [28]:
#QC
y_act.Encoded_SKU_ID.nunique(),y_pred_null.Encoded_SKU_ID.nunique(),y_pred_best.Encoded_SKU_ID.nunique()

(545, 545, 545)

In [29]:
rmse_val_null,rmse_null_best = rmse(y_act,y_pred_null),rmse(y_act,y_pred_best)
rmse_val_null,rmse_null_best

(6.59603916258372, 1.0406568873731874)

In [30]:
(rmse_val_null-rmse_null_best)/rmse_val_null

0.8422300320355354

In [31]:
rmse_all_preds["min_period"].value_counts()

rmse_du_15      222
rmse_du_STL      94
rmse_du_max      41
rmse_du_30       36
rmse_du_null     35
rmse_du_60       30
rmse_du_90       27
rmse_du_45       22
rmse_du_holt     21
rmse_du_MSTL     17
Name: min_period, dtype: int64

### Analysis

In [32]:
rmse_all_preds.head()

,rmse_du_15,mean_du_15,pct_rmse_15,rmse_du_30,mean_du_30,pct_rmse_30,rmse_du_45,mean_du_45,pct_rmse_45,rmse_du_60,...,pct_rmse_MSTL,rmse_du_null,mean_du_null,pct_rmse_null,rmse_du_holt,mean_du_holt,pct_rmse_holt,min_rmse,argmin_period,min_period
Encoded_SKU_ID,,,,,,,,,,,,,,,,,,,,,
82,4.722325,0.428571,11.018759,4.840344,0.428571,11.294136,1.102108,0.428571,2.571586,1.419381,...,12.555916,3.605551,0.428571,8.412953,9.753826,0.428571,22.758928,0.911043,5,rmse_du_max
568,4.097124,4.285714,0.955996,3.447670,4.285714,0.804456,3.652836,4.285714,0.852329,3.871231,...,0.631231,5.070926,4.285714,1.183216,3.024439,4.285714,0.705702,2.705275,7,rmse_du_MSTL
317,3.766488,0.857143,4.394235,1.362770,0.857143,1.589899,1.784957,0.857143,2.082450,2.104417,...,2.943792,1.069045,0.857143,1.247219,2.363641,0.857143,2.757581,1.069045,8,rmse_du_null
95,3.668447,1.285714,2.853236,1.698949,1.285714,1.321405,3.991151,1.285714,3.104228,3.931648,...,1.971178,1.309307,1.285714,1.018350,1.720612,1.285714,1.338254,1.192371,6,rmse_du_STL
340,3.668447,0.714286,5.135825,2.077172,0.714286,2.908041,2.329929,0.714286,3.261901,1.558387,...,1.360928,0.925820,0.714286,1.296148,1.092959,0.714286,1.530142,0.727766,5,rmse_du_max


In [33]:
train_test.head()

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS,max_SALES_DATE,delta_days,validation,validation_clean,if_promo,if_competitor,min_sales_price,comp_price_clean,price_delta
0,1,2017-09-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,?,Fully-Stocked,1,2022-07-31,1767,False,True,0,0,89.99,NaN,-89.99
1,2,2017-11-24,CAMERA BATTERIES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,39.99,?,?,Fully-Stocked,18,2022-07-31,1710,False,True,0,0,39.99,NaN,-39.99
2,3,2019-11-28,SO PARTS,MOBILE ACCESSORIES,ML - Connected Car,VP - Electrify & Car,49.99,?,46.98,Constrained,7,2022-07-31,976,False,True,0,1,49.99,46.98,3.01
3,4,2021-04-21,SO RANGEHOODS REG,KITCHEN VENTILATION,ML - Kitchen,VP - Major Appliances,549.99,?,498,Fully-Stocked,6,2022-07-31,466,False,False,0,1,549.99,498.00,51.99
4,1,2018-10-27,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,?,Fully-Stocked,7,2022-07-31,1373,False,True,0,0,89.99,NaN,-89.99


In [49]:
rst = rmse_all_preds.reset_index(drop= False)[['Encoded_SKU_ID','min_rmse','min_period','rmse_du_null']]


In [50]:
rst.rename({'min_period':'model'})

,Encoded_SKU_ID,min_rmse,min_period,rmse_du_null
0,82,0.911043,rmse_du_max,3.605551
1,568,2.705275,rmse_du_MSTL,5.070926
2,317,1.069045,rmse_du_null,1.069045
3,95,1.192371,rmse_du_STL,1.309307
4,340,0.727766,rmse_du_max,0.925820
...,...,...,...,...
540,558,1.069045,rmse_du_15,1.069045
541,559,3.098009,rmse_du_15,3.207135
542,560,0.534522,rmse_du_15,0.534522
543,566,2.507133,rmse_du_15,2.507133


In [38]:
train_test.head()

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS,max_SALES_DATE,delta_days,validation,validation_clean,if_promo,if_competitor,min_sales_price,comp_price_clean,price_delta
0,1,2017-09-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,?,Fully-Stocked,1,2022-07-31,1767,False,True,0,0,89.99,NaN,-89.99
1,2,2017-11-24,CAMERA BATTERIES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,39.99,?,?,Fully-Stocked,18,2022-07-31,1710,False,True,0,0,39.99,NaN,-39.99
2,3,2019-11-28,SO PARTS,MOBILE ACCESSORIES,ML - Connected Car,VP - Electrify & Car,49.99,?,46.98,Constrained,7,2022-07-31,976,False,True,0,1,49.99,46.98,3.01
3,4,2021-04-21,SO RANGEHOODS REG,KITCHEN VENTILATION,ML - Kitchen,VP - Major Appliances,549.99,?,498,Fully-Stocked,6,2022-07-31,466,False,False,0,1,549.99,498.00,51.99
4,1,2018-10-27,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,?,Fully-Stocked,7,2022-07-31,1373,False,True,0,0,89.99,NaN,-89.99


In [51]:
data_clean= train_test.merge(rst, on ='Encoded_SKU_ID')

In [44]:
train_test[train_test['validation_clean'] == True].shape

(801305, 20)

In [40]:
data_clean.shape

(801305, 22)

In [52]:
data_clean.to_csv('/Users/yibeihu/Desktop/BB_rst.csv')

In [47]:
data_clean.head()

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,...,delta_days,validation,validation_clean,if_promo,if_competitor,min_sales_price,comp_price_clean,price_delta,min_rmse,min_period
0,1,2017-09-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,?,Fully-Stocked,...,1767,False,True,0,0,89.99,NaN,-89.99,0.755929,rmse_du_30
1,1,2018-10-27,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,?,Fully-Stocked,...,1373,False,True,0,0,89.99,NaN,-89.99,0.755929,rmse_du_30
2,1,2017-08-27,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,?,Fully-Stocked,...,1799,False,True,0,0,89.99,NaN,-89.99,0.755929,rmse_du_30
3,1,2021-03-08,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,69,Fully-Stocked,...,510,False,True,0,1,89.99,69.0,20.99,0.755929,rmse_du_30
4,1,2019-11-04,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,69,Fully-Stocked,...,1000,False,True,0,1,89.99,69.0,20.99,0.755929,rmse_du_30
